# 🏛️ LexGuard - Fine-tuning Llama-3-8B for Legal Metadata Extraction

This notebook fine-tunes **Llama-3-8B** on 500 labeled legal contracts from the CUAD dataset.

### Steps:
1. Install Unsloth
2. Upload `finetune_data.jsonl`
3. Load model & train
4. Evaluate on test set
5. Save model

> ⚠️ **Before running:** Go to `Runtime → Change runtime type → T4 GPU`

## Step 1: Install Unsloth

In [ ]:
%%capture
!pip install unsloth
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

## Step 2: Upload your `finetune_data.jsonl` file

Run this cell, then click **Choose Files** and select `finetune_data.jsonl` from your computer.

In [ ]:
from google.colab import files
uploaded = files.upload()
print('✅ File uploaded:', list(uploaded.keys()))

## Step 3: Load and Split Dataset (70% Train / 30% Test)

In [ ]:
from datasets import load_dataset

# Load the JSONL file
full_dataset = load_dataset('json', data_files='finetune_data.jsonl', split='train')
print(f'Total examples: {len(full_dataset)}')

# Split into 70% train / 30% test
split = full_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split['train']
test_dataset  = split['test']

print(f'Train set: {len(train_dataset)} examples')
print(f'Test set : {len(test_dataset)} examples')

## Step 4: Load Model with Unsloth (4-bit quantized for T4 GPU)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Supports up to 4096 tokens
dtype = None           # Auto-detect (float16 for T4)
load_in_4bit = True    # Required for T4 GPU (16GB VRAM)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Meta-Llama-3-8B-Instruct-bnb-4bit',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print('✅ Model loaded!')

## Step 5: Apply LoRA (makes training fast & memory-efficient)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                    # LoRA rank (16 is a good balance)
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj',
                      'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)
print('✅ LoRA applied!')

## Step 6: Format Dataset for Training

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = 'llama-3',
)

def formatting_prompts_func(examples):
    convos = examples['messages']
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
             for convo in convos]
    return {'text': texts}

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
test_dataset  = test_dataset.map(formatting_prompts_func, batched=True)
print('✅ Dataset formatted!')
print('Sample:')
print(train_dataset[0]['text'][:500])

## Step 7: Train! 🚀

This will take **2-4 hours on a free T4 GPU**. You can leave it running.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = 'text',
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,       # Train for 2 full passes over the data
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 42,
        output_dir = 'lexguard_outputs',
        report_to = 'none',
    ),
)

print('🚀 Starting training...')
trainer_stats = trainer.train()
print('✅ Training complete!')
print(f'   Training loss: {trainer_stats.training_loss:.4f}')

## Step 8: Evaluate with LLM Judge (Groq API)

Thay vì so sánh exact string (quá khắt khe), mình dùng **LLM judge** để đánh giá ngữ nghĩa.
Ví dụ: `"5/8/14"` và `"May 8, 2014"` sẽ được judge coi là **match**.

> Bạn cần **Groq API key** miễn phí tại [console.groq.com](https://console.groq.com)
> Điền API key vào biến `GROQ_API_KEY` ở cell bên dưới.

In [ ]:
!pip install groq -q
print("Groq installed")

In [ ]:
import json, time
import torch
from groq import Groq
from unsloth import FastLanguageModel

# =====================================================
# TODO: Nhập Groq API key của bạn vào đây
GROQ_API_KEY = "gsk_YOUR_KEY_HERE"
# =====================================================

groq_client = Groq(api_key=GROQ_API_KEY)
JUDGE_MODEL  = "openai/gpt-oss-120b"
JUDGE_SYSTEM = (
    "You are an expert evaluator for legal contract metadata extraction. "
    "Given extracted values and ground truth values, decide if they semantically match. "
    "Minor formatting differences (date formats, capitalization, brackets) should be IGNORED. "
    'Output ONLY valid JSON where each key is the field name: '
    '{"Field Name": {"match": true/false, "reasoning": "one line"}}'
)

def judge_all_fields(extracted, ground_truth):
    lines = ["Evaluate each field extraction below:\n"]
    for field, gt_val in ground_truth.items():
        pred_val = extracted.get(field, "None mentioned")
        lines.append(f'Field: "{field}"\n  Extracted   : {pred_val}\n  Ground Truth: {gt_val}\n')
    user_msg = "\n".join(lines)
    for attempt in range(3):
        try:
            resp = groq_client.chat.completions.create(
                model=JUDGE_MODEL,
                messages=[
                    {"role": "system", "content": JUDGE_SYSTEM},
                    {"role": "user",   "content": user_msg}
                ],
                response_format={"type": "json_object"},
                temperature=0,
            )
            return json.loads(resp.choices[0].message.content)
        except Exception as e:
            print(f"  Judge error (attempt {attempt+1}): {e}. Waiting 3s...")
            time.sleep(3)
    return {}

# ── Main Evaluation Loop ──────────────────────────────────────────────
FastLanguageModel.for_inference(model)

num_samples    = min(20, len(test_dataset))
total_matches  = 0
total_mismatches = 0

print(f"Evaluating {num_samples} test examples with LLM Judge...\n")

for i in range(num_samples):
    example  = test_dataset[i]
    messages = example["messages"]
    try:
        ground_truth = json.loads(messages[-1]["content"])
    except Exception:
        continue

    doc_name = ground_truth.get("Document Name", f"Example {i+1}")
    print(f"[{i+1}/{num_samples}] {str(doc_name)[:60]}")

    # Generate model output
    prompt_messages = messages[:-1]
    inputs = tokenizer.apply_chat_template(
        prompt_messages, tokenize=True,
        add_generation_prompt=True, return_tensors="pt"
    ).to("cuda")
    with torch.no_grad():
        out = model.generate(
            input_ids=inputs, max_new_tokens=2048,
            temperature=0.1, do_sample=False
        )
    generated = tokenizer.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True)

    # Parse model JSON
    try:
        s = generated.find("{")
        e = generated.rfind("}") + 1
        predicted = json.loads(generated[s:e])
    except Exception:
        print("  Could not parse model JSON, skipping...")
        total_mismatches += len(ground_truth)
        continue

    # Ask LLM Judge
    judge_results = judge_all_fields(predicted, ground_truth)
    ex_matches    = sum(1 for r in judge_results.values()
                        if isinstance(r, dict) and r.get("match", False))
    ex_mismatches = len(ground_truth) - ex_matches
    total_matches    += ex_matches
    total_mismatches += ex_mismatches
    print(f"  Matches: {ex_matches}  |  Mismatches: {ex_mismatches}")

total_fields = total_matches + total_mismatches
accuracy = (total_matches / total_fields * 100) if total_fields > 0 else 0
print(f"\nFinal Accuracy (LLM Judge): {total_matches}/{total_fields} = {accuracy:.1f}%")
print("(Semantic accuracy -- much more meaningful than exact string matching!)")


## Step 9: Save the fine-tuned model

Choose one of the options below:

In [ ]:
# Option A: Save as GGUF (for running locally on Mac with Ollama / LM Studio)
# This creates a file you can download and use on your Mac without internet!
model.save_pretrained_gguf('lexguard_model', tokenizer, quantization_method='q4_k_m')
print('✅ GGUF model saved to lexguard_model/')
print('   Download this folder and use with Ollama or LM Studio on your Mac.')

In [ ]:
# Option B: Push to Hugging Face Hub (to use as an API like Groq)
# You need a free HuggingFace account and token from https://huggingface.co/settings/tokens

# from huggingface_hub import login
# login(token='YOUR_HF_TOKEN_HERE')

# model.push_to_hub('your-username/lexguard-llama3-8b', tokenizer=tokenizer)
# print('✅ Model pushed to HuggingFace Hub!')

print('Uncomment the code above and add your HuggingFace token to use this option.')

## ✅ Done!

Your **LexGuard** model is now fine-tuned for legal metadata extraction!

**Next steps:**
- Download the GGUF file → use with **Ollama** on your Mac (free, offline!)
- Or push to HuggingFace Hub → replace Groq API in `pipeline.py` with your new model